In [1]:
from transformers import AutoProcessor, AutoTokenizer, Gemma3ForConditionalGeneration

model_id = "google/gemma-3-27b-it"
save_path = "./gemma-3-27b-it"

c:\Users\sduailab\Aimat\research_1\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Gemma3ForConditionalGeneration.from_pretrained(model_id)
model.save_pretrained(save_path)

processor = AutoProcessor.from_pretrained(model_id)
processor.save_pretrained(save_path)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.save_pretrained(save_path)

Loading checkpoint shards: 100%|██████████| 12/12 [00:38<00:00,  3.22s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


('./gemma-3-27b-it\\tokenizer_config.json',
 './gemma-3-27b-it\\special_tokens_map.json',
 './gemma-3-27b-it\\chat_template.jinja',
 './gemma-3-27b-it\\tokenizer.json')

In [3]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import torch

local_path = "./gemma-3-27b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    local_path,
    device_map="auto", 
    torch_dtype=torch.bfloat16
).eval()

processor = AutoProcessor.from_pretrained(local_path)

Loading checkpoint shards: 100%|██████████| 22/22 [01:27<00:00,  3.96s/it]


In [4]:
from PIL import Image
from transformers import GenerationConfig

gen_config = GenerationConfig.from_pretrained("./gemma-3-27b-it")
gen_config.cache_implementation = "dynamic" 

messages = [
    {"role": "system", "content": [{"type": "text", "text": "You are smart assistant"}]},
    {"role": "user", "content": [
        {"type": "image", "image": Image.open("C:/Users/sduailab/Aimat/research_1/C._Ronaldo_-_Ballon_d'Or_2014.jpg")}, 
        {"type": "text", "text": "What do you see?"}
    ]}
]

inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device)


output = model.generate(**inputs, generation_config=gen_config, max_new_tokens=200)

print(processor.decode(output[0], skip_special_tokens=True))


user
You are smart assistant





What do you see?
model
Here's what I see in the image:

**Main Subject:**

*   Cristiano Ronaldo is the central figure. He's a well-known Portuguese professional footballer.
*   He is holding a Ballon d'Or trophy. This is an annual football award presented to the best male player in the world.

**Attire:**

*   He's wearing a white Real Madrid football kit, including the jersey and shorts. The number "7" is clearly visible on his shorts.
*   He has long sleeves rolled up.

**Background:**

*   The background is a grassy field, likely a football pitch.
*   The background is slightly blurred, which focuses attention on Ronaldo.

**Overall Impression:**

The photo appears to be taken shortly after Ronaldo received the Ballon d'Or, and he's likely on the field acknowledging fans or celebrating.


In [5]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

loader = PyPDFLoader("C:/Users/sduailab/Aimat/research_1/MY20-AMG_GT_Group_WebPDF_091719.pdf")  
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(texts, embedding, persist_directory="./rag_db")
retriever = vectordb.as_retriever()

C:\Users\sduailab\AppData\Local\Temp\ipykernel_24488\1052465098.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
from transformers import AutoTokenizer, Gemma3ForConditionalGeneration
import torch

model = Gemma3ForConditionalGeneration.from_pretrained(
    "./gemma-3-27b-it", torch_dtype=torch.bfloat16, device_map="auto"
).eval()
tokenizer = AutoTokenizer.from_pretrained("./gemma-3-27b-it")

Loading checkpoint shards: 100%|██████████| 22/22 [00:39<00:00,  1.80s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [7]:
query = "Что за двигатель в AMG GT?"

docs = retriever.get_relevant_documents(query)
context = "\n".join([doc.page_content for doc in docs[:3]])

prompt = f"Контекст:\n{context}\n\nВопрос: {query}\nОтвет:"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=200)

answer = tokenizer.decode(output[0], skip_special_tokens=True)
print(answer)


C:\Users\sduailab\AppData\Local\Temp\ipykernel_24488\1432690455.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


Контекст:
AMG GT R PRO COUPE
Handcrafted 577-hp AMG V8 
biturbo engine
designo  Selenite Grey Magno  
Grey 19"/20" AMG® forged  
 twin 5 -spoke wheels1
33
AMG GT R PRO COUPE
Handcrafted 577-hp AMG V8 
biturbo engine
designo  Selenite Grey Magno  
Grey 19"/20" AMG® forged  
 twin 5 -spoke wheels1
33
AMG GT R PRO COUPE
Handcrafted 577-hp AMG V8 
biturbo engine
designo  Selenite Grey Magno  
Grey 19"/20" AMG® forged  
 twin 5 -spoke wheels1
33

Вопрос: Что за двигатель в AMG GT?
Ответ: Двигатель в AMG GT - это 577-сильный AMG V8 с двойным турбонаддувом.

